# Simple RAG for GitHub issues using Hugging Face Zephyr and LangChain

_Authored by: [Maria Khalusova](https://github.com/MKhalusova)_

This notebook demonstrates how you can quickly build a RAG (Retrieval Augmented Generation) for a project's GitHub issues using [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta) model, and LangChain.


**What is RAG?**

RAG is a popular approach to address the issue of a powerful LLM not being aware of specific content due to said content not being in its training data, or hallucinating even when it has seen it before. Such specific content may be proprietary, sensitive, or, as in this example, recent and updated often.

If your data is static and doesn't change regularly, you may consider fine-tuning a large model. In many cases, however, fine-tuning can be costly, and, when done repeatedly (e.g. to address data drift), leads to "model shift". This is when the model's behavior changes in ways that are not desirable.

**RAG (Retrieval Augmented Generation)** does not require model fine-tuning. Instead, RAG works by providing an LLM with additional context that is retrieved from relevant data so that it can generate a better-informed response.

Here's a quick illustration:

![RAG diagram](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/rag-diagram.png)

* The external data is converted into embedding vectors with a separate embeddings model, and the vectors are kept in a database. Embeddings models are typically small, so updating the embedding vectors on a regular basis is faster, cheaper, and easier than fine-tuning a model.

* At the same time, the fact that fine-tuning is not required gives you the freedom to swap your LLM for a more powerful one when it becomes available, or switch to a smaller distilled version, should you need faster inference.

Let's illustrate building a RAG using an open-source LLM, embeddings model, and LangChain.

First, install the required dependencies:

## Prepare the data


In [1]:
!pip install -qU pypdf

In [25]:
from langchain_community.document_loaders import PyPDFLoader

pages = []
for fn in os.listdir("data/"):
    loader = PyPDFLoader(f"data/{fn}")
    async for page in loader.alazy_load():
        pages.append(page)

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=30)

chunked_docs = splitter.split_documents(pages)

In [6]:
!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 13.8 MB/s eta 0:00:00a 0:00:01


In [6]:
import requests
r = requests.get('https://www.hp.com/us-en/home.html')
r = requests.get('https://en.wikipedia.org/wiki/HP_Inc.')
content = r.text
with open('sample.html','w',encoding="utf-8") as f:
    f.write(content)


In [1]:
from langchain_community.document_loaders import BSHTMLLoader
loader = BSHTMLLoader("sample.html", bs_kwargs={"features":"lxml"}) #

docs = loader.load()

The content of individual GitHub issues may be longer than what an embedding model can take as input. If we want to embed all of the available content, we need to chunk the documents into appropriately sized pieces.

The most common and straightforward approach to chunking is to define a fixed size of chunks and whether there should be any overlap between them. Keeping some overlap between chunks allows us to preserve some semantic context between the chunks. The recommended splitter for generic text is the [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter), and that's what we'll use here.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

In [23]:
chunked_docs[0]

Document(metadata={'source': 'data/HP Z6 G4 Workstation.pdf', 'page': 0}, page_content='1\nThe HP Z6 G4 is the successor to the HP Z640 personal workstation.  \nIts architecture introduces several new functionalities and technologies. \nThese include all new Intel® memory architecture, Intel®-integrated 10 \nGbE, I/O slot improvements and enhanced performance.\nHP Z6 G4 WORKSTATION\nCHASSIS AND SYSTEM HIGHLIGHTS\nMore power and configurability! HP is proud to introduce the all new HP Z6 G4 Workstation. This surprisingly small')

## Create the embeddings + retriever

Now that the docs are all of the appropriate size, we can create a database with their embeddings.

To create document chunk embeddings we'll use the `HuggingFaceEmbeddings` and the [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5) embeddings model. There are many other embeddings models available on the Hub, and you can keep an eye on the best performing ones by checking the [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).


To create the vector database, we'll use `FAISS`, a library developed by Facebook AI. This library offers efficient similarity search and clustering of dense vectors, which is what we need here. FAISS is currently one of the most used libraries for NN search in massive datasets.

We'll access both the embeddings model and FAISS via LangChain API.

In [27]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

import os
#os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))


We need a way to return(retrieve) the documents given an unstructured query. For that, we'll use the `as_retriever` method using the `db` as a backbone:
- `search_type="similarity"` means we want to perform similarity search between the query and documents
- `search_kwargs={'k': 4}` instructs the retriever to return top 4 results.


In [28]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

The vector database and retriever are now set up, next we need to set up the next piece of the chain - the model.

## Load quantized model

For this example, we chose [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a small but powerful model.

With many models being released every week, you may want to substitute this model to the latest and greatest. The best way to keep track of open source LLMs is to check the [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

To make inference faster, we will load the quantized version of the model:

In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "nvidia/Mistral-NeMo-Minitron-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Setup ML Flow

In [30]:
import mlflow
mlflow.set_experiment("Mistral-NeMo-Minitron-8B-Instruct RAG")
mlflow.langchain.autolog(
    # These are optional configurations to control what information should be logged automatically (default: False)
    # For the full list of the arguments, refer to https://mlflow.org/docs/latest/llms/langchain/autologging.html#id1
    log_models=True,
    log_input_examples=True,
    log_model_signatures=True,
)

# Configure retriever load for ML Flow
persist_dir = "faiss_index"
db.save_local(persist_dir)
def load_retriever(persist_directory):
    embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
    vectorstore = FAISS.load_local(persist_directory, embeddings)
    return vectorstore.as_retriever()

# Log the retriever with the loader function
with mlflow.start_run() as run:
    logged_model = mlflow.langchain.log_model(
        db.as_retriever(),
        artifact_path="retriever",
        loader_fn=load_retriever,
        persist_dir=persist_dir,
    )

/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/utils/docstring_utils.py:421: UserWarning: The 'langchain' MLflow Models integration is known to be compatible with the following package version ranges: ``0.0.244`` -  ``0.2.11``. MLflow Models integrations with langchain may not succeed when used with package versions outside of this range.
  _do_version_compatibility_warning(notice)
/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/utils/docstring_utils.py:421: UserWarning: The 'langchain' MLflow Models integration is known to be compatible with the following package version ranges: ``0.0.244`` -  ``0.2.11``. MLflow Models integrations with langchain may not succeed when used with package versions outside of this range.
  _do_version_compatibility_warning(notice)
2024/12/04 01:40:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpnsmdgoy6/model, flavor: langchain). Fall back to return ['langc

## Setup the LLM chain

Finally, we have all the pieces we need to set up the LLM chain.

First, create a text_generation pipeline using the loaded model and its tokenizer.

Next, create a prompt template - this should follow the format of the model, so if you substitute the model checkpoint, make sure to use the appropriate formatting.

In [31]:
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Note: _You can also use `tokenizer.apply_chat_template` to convert a list of messages (as dicts: `{'role': 'user', 'content': '(...)'}`) into a string with the appropriate chat format._


Finally, we need to combine the `llm_chain` with the retriever to create a RAG chain. We pass the original question through to the final generation step, as well as the retrieved context docs:

In [32]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


## Compare the results

Let's see the difference RAG makes in generating answers to the library-specific questions.

In [33]:
question = "what is the best hp computer model for training large data science models?"

First, let's see what kind of answer we can get with just the model itself, no context added:

In [ ]:
with mlflow.start_run() as run:
    logged_model = mlflow.langchain.log_model(
            llm_chain,
            artifact_path="retrieval_qa",
            loader_fn=load_retriever,
            persist_dir=persist_dir,
    )
    print(llm_chain.invoke({"context":"", "question": question}))

/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/utils/docstring_utils.py:421: UserWarning: The 'langchain' MLflow Models integration is known to be compatible with the following package version ranges: ``0.0.244`` -  ``0.2.11``. MLflow Models integrations with langchain may not succeed when used with package versions outside of this range.
  _do_version_compatibility_warning(notice)
/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/utils/docstring_utils.py:421: UserWarning: The 'langchain' MLflow Models integration is known to be compatible with the following package version ranges: ``0.0.244`` -  ``0.2.11``. MLflow Models integrations with langchain may not succeed when used with package versions outside of this range.
  _do_version_compatibility_warning(notice)
2024/12/04 01:40:42 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
Setting `pad_token_id` to `eos_t

As you can see, the model interpreted the question as one about physical computer adapters, while in the context of PEFT, "adapters" refer to LoRA adapters.
Let's see if adding context from GitHub issues helps the model give a more relevant answer:

In [ ]:
with mlflow.start_run() as run:
    logged_model = mlflow.langchain.log_model(
            rag_chain,
            artifact_path="retrieval_qa",
            loader_fn=load_retriever,
            persist_dir=persist_dir,
    )
    print(rag_chain.invoke(question))

As we can see, the added context, really helps the exact same model, provide a much more relevant and informed answer to the library-specific question.

Notably, combining multiple adapters for inference has been added to the library, and one can find this information in the documentation, so for the next iteration of this RAG it may be worth including documentation embeddings.